In [2]:
import os
import sys
import pandas as pd
import yfinance as yf
from lib.auxiliares.esReal import es_real
from lib.auxiliares.VaR import var_vol
from lib.auxiliares.ES import expected_shortfall
from lib.auxiliares.simulateReturns import simulate_returns
from lib.volatilidades.volatilities import *
from lib.volatilidades.rolling_forecast import *
from lib.backtest.RidgeBacktest import *

In [3]:
confidence_level = 0.975
indexes = ['SAN.MC', 'BBVA.MC', 'SAB.MC', '^IBEX', 'BBVAE.MC', 'XTC5.MI', 'EURUSD=X']
first_historical_date = '2021-07-31'
start_date = '2024-06-30'
end_date = '2024-07-30'
horizon = 1

In [4]:
data = yf.download("SAN.MC", first_historical_date, end_date, progress=False)
data['Log Returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))

In [5]:
es_real(data, confidence_level, start_date, end_date)

(-0.03261981095941505, 1)

In [6]:
std_volatility = calculate_std_volatility(data)
ewma_volatility = calculate_ewma_volatility(data)
gjrgarch_volatility = calculate_gjrgarch_volatility(data)

In [7]:
forecast = roll_perceptron_forecast(std_volatility, start_date, end_date, horizon)

TypeError: roll_perceptron_forecast() missing 3 required positional arguments: 'global_counter', 'lock', and 'total_tasks'

In [1]:
forecast

NameError: name 'forecast' is not defined

In [7]:
var = var_vol(forecast, confidence_level)
es = expected_shortfall(forecast, confidence_level)
real_returns = data['Log Returns'][start_date:end_date].dropna()

In [8]:
es_test = EStestRidge(real_returns, lambda: np.random.normal(loc=0, scale=forecast['VOLATILITY'].values, size=len(forecast['VOLATILITY'].values)),
                                      1 - confidence_level,
                                      var, es, 1000, 1 - confidence_level)

In [9]:
es_test.salida()

(2, -0.030448353693478838)

In [10]:
es_test.print()

----------------------------------------------------------------
        Ridge Expected Shortfall Test by Simulation             
----------------------------------------------------------------
Number of observations: 21
Number of VaR breaches: 2
Expected number of VaR breaches: 0.5250000000000005
ES Statistic: -2.6608985409391774
Expected ES Statistic under the null hypothesis: 0
Critical Value at α = 0.025000000000000022: -3.0931582497850005
p-value: 0.06
Number of Monte Carlo simulations: 1000
----------------------------------------------------------------
